# The Skew-T Log-p Diagram


In [ ]:
from datetime import datetime

import matplotlib.pylab as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import metpy.calc as mpcalc
from metpy.plots import Hodograph, SkewT    # NEW
from metpy.units import units, pandas_dataframe_to_unit_arrays
import numpy as np
from siphon.simplewebservice.wyoming import WyomingUpperAir 

In [ ]:
date = datetime(2011, 4, 27, 0)
station = 'BMX'
df = WyomingUpperAir.request_data(date, station)
data_with_units = pandas_dataframe_to_unit_arrays(df)

# Only want to pull in data up to 100 mb, we can 
# use a boolean array to choose on the desired values
i_p100 = data_with_units['pressure'] >= 100 * units.hPa

# Subset pressure and read in other data to only 100 mb
p = data_with_units['pressure'][i_p100]
T = data_with_units['temperature'][i_p100]
Td = data_with_units['dewpoint'][i_p100]
u = data_with_units['u_wind'][i_p100]
v = data_with_units['v_wind'][i_p100]

## Plotting the Skew-T

MetPy has the ability to make a few specialty plots (with matplotlib as a backend). The skew-T plots are relatively easy to do with only a few lines to plot the key variables and lines on the plot. The main Class that we have to draw a sounding is the `SkewT` object. Here we have rotated the axes and have the ability to draw lines with the `skew.plot()` method, draw wind barbs with the `skew.plot_barbs` method and even plot the special lines of dry adiabats, moist adiabats, and mixing ratio.

https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.SkewT.html#metpy.plots.SkewT

In [ ]:
fig = plt.figure(figsize=(10, 10))

# Change default to be better for skew-T
# We imported SkewT from metpy.plots above.
skew = SkewT(fig, rotation=45)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, T, 'r', lw=2)
skew.plot(p, Td, 'g', lw=2)
#skew.plot_barbs(p, u, v)

# Add the relevant special lines
#skew.plot_dry_adiabats()
#skew.plot_moist_adiabats()
#skew.plot_mixing_lines()

#skew.ax.set_ylim(1000,100)
#skew.ax.set_xlim(-50,50)

# Set title using the date and station variables directly
#plt.title(f'{date:%H UTC %d %B %Y} at K{station}')

# Show the plot
plt.tight_layout()
plt.show()

### Refined Skew-T Plots

Draw better dry adiabats, moist adiabats, and mixing ratio lines (my defaults).

https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.SkewT.html#metpy.plots.SkewT

Add text and markers to the plot using the `text` and `plot` methods available on the skew-T object main axes `skew.ax.text` and `skew.ax.plot`. Accessing these base-level Matplotlib methods will work just the same as if you were plotting a typical x-y plot, but instead using pressure and temperature to plot appropriately on our skewed axis. Note that the ordering is reversed from our `SkewT` class object in that the x-value goes first (temperature) and the y-value (pressure) goes second.

In [ ]:
# Copy code from previous code cell.
# Subset wind barbs so only every third barb is shown. e.g. p[::3]


# metpy also has the resample_nn_1d() function to return indices close to each element in an array.
#interval = np.logspace(2, 3, 40) * units.hPa
#idx = mpcalc.resample_nn_1d(p, interval)
#skew.plot_barbs(pressure=p[idx], u=u[idx], v=v[idx])


# Use arguments to adjust the appearance of adiabats and mixing ratio lines.
#skew.plot_dry_adiabats(t0=np.arange(-60,240,10) * units.degC, color='darkorange', alpha=0.25)
#skew.plot_mixing_lines(pressure=np.arange() * units.hPa)

# Add some labels for 850 hPa.
skew.ax.text(-9,850,'850 hPa',va='center')
skew.ax.plot(-11,850,marker='_',ms=16,markeredgewidth=2)

## Calculate Parcel Profile

Now we want to use the MetPy function `parcel_profile` to calculated the surfaced-baed CAPE (SBCAPE). We'll need to feed the function with all of the pressure values along with a single starting temperature and dewpoint value. The starting temperature and dewpoint values should be the first value available in each array.

https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.parcel_profile.html#metpy.calc.parcel_profile

In [ ]:
parcel_temp = mpcalc.parcel_profile(p, T[0], Td[0])

In [ ]:
print(parcel_temp)

## Add Parcel Profile to Skew-T and Fill CAPE Area

We can start from our improved skew-T that we generated above, then add a new `skew.plot()` to plot the parcel temperature profile for each level in the pressure variable. Additionally we can use the method available in a `SkewT()` object to shade CAPE values `skew.shade_cape()`. A similar process works for CIN using the appropriate shade method.

https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.SkewT.html#metpy.plots.SkewT.shade_cape

In [ ]:
# Copy skew-T code from above. Remove lines for the 850hPa marker and text.
# Add another skew.plot() for parcel_temp.
# Adjust title so station is on the left and date is on teh right.
# Use skew.shade_cape(p,T,parcel_temp)

## Hodograph Addition
We can also use the `Hodograph()` plot functionality to add that to our skew-T diagram. Here we'll demonstrate adding it as an inset axes within out skew-T plot itself. Note that hodographs are plotted by height (not pressure) and we'll add a few nice elements to make our hodograph more interpretable.

https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.Hodograph.html

In [ ]:
# Copy code from above.

# Create a hodograph
hght = data_with_units['height'][ip100]

ax_hod = inset_axes(skew.ax, '25%', '25%', loc=1, borderpad=0.8)
h = Hodograph(ax_hod, component_range=80)
h.add_grid(increment=20)
h.plot_colormapped(u,v,hght)

# Can subset hodograph from surface to 6 km.
iz6km = hght <= 5 * units.km
hght_6km = hght[iz6km]